In [8]:
from sagemaker.jumpstart.model import JumpStartModel

model_id = "huggingface-llm-gemma-7b-instruct"


In [9]:
accept_eula = True

In [10]:
instance_type = "ml.g5.12xlarge"

model = JumpStartModel(model_id=model_id, instance_type=instance_type)

In [11]:
origin_model = model.deploy(accept_eula=accept_eula)

-------------------------------------!

In [41]:
import sacrebleu
from rouge import Rouge
import pandas as pd
import re
import json

def extract_referral_content(text):
    # 使用正则表达式匹配倒数第二个和最后一个双引号之间的内容
    pattern = r'"([^"]*)"\s*[^"]*$'
    match = re.search(pattern, text)
    # print(f"Extracting referral_content from text:\n{text}")  # 调试信息
    if match:
        return match.group(1)
    else:
        return "extract failure"

# def extract_referral_content(prediction_text):
#     try:
#         # 檢查不同格式的 referral_content 並提取值
#         if '**referral_content:**' in prediction_text:
#             match = re.search(r'\*\*referral_content:\*\*\s*"([^"]+)"', prediction_text)
#         elif 'referral_content:' in prediction_text:
#             match = re.search(r'referral_content:\s*"([^"]+)"', prediction_text)
#         else:
#             match = re.search(r'"referral_content":\s*"([^"]+)"', prediction_text)

#         if match:
#             return match.group(1)
#         else:
#             return "extract failure"
#     except Exception as err:
#         return "extract failure"
    
def extract_is_papilledema(prediction_text):
    try:
        # 使用正则表达式从生成的文本中提取最后一个 is_Papilledema 的值，不管中间有什么符号
        is_papilledema_matches = list(re.finditer(r'is_Papilledema.*?(True|False)', prediction_text, re.IGNORECASE))
        if is_papilledema_matches:
            last_match = is_papilledema_matches[-1]
            return "True" if last_match.group(1).lower() == 'true' else "False"
        else:
            return "extract failure"
    except Exception as err:
        return "extract failure"
    
# def extract_is_papilledema(prediction_text):
#     try:
        
#         match = re.search(r'(?:\*\*|)is_Papilledema(?:\*\*|):\s*"([^"]+)"', prediction_text)
#         if match:
#             return match.group(1)
#         else:
#             return "extract failure"
#     except Exception as err:
#         return "extract failure"

    
def evaluate_testjsonl_with_gemma(reference_path, csv_file_path):
    
    test_data_json = []
    
    with open(reference_path, 'r', encoding='utf-8') as f:
        for line in f:
              test_data_json.append(json.loads(line.strip()))
    
    rouge_score_list = []
    bleu_score_list = []
              
    rouge = Rouge()
              
    prediction_list = []
              
    for single_test in test_data_json:
        instruction = single_test["instruction"]
        whole_letter = single_test["whole_letter"]
        referral_content = single_test["referral_content"]
        is_papilledema = single_test["is_Papilledema"]
        prompt = f"{instruction}\n\n###\n\n{whole_letter}\n\n###"
        response = origin_model.predict({'inputs': prompt, 'parameters': {'max_new_tokens': 256}})
        # response = origin_model.predict({prompt})

        reference_text = referral_content
        # print(response[0])
        try:
            # print("response: " + response)
            # response = reponse[0] if isinstance(response, list) else response
            # print("response: " + response)
            # prediction_dict = json.loads(response["generated_text"].strip())
            prediction_text = response[0]["generated_text"].strip()
            # print(prediction_text)
            
            
            extracted_referral_content = extract_referral_content(prediction_text)
            # print(extracted_referral_content)
            
            extracted_is_papilledema = extract_is_papilledema(prediction_text)
            # print(extracted_is_papilledema)
            
            
            single_test["predict_referral_content"] = extracted_referral_content
            single_test["predict_is_papilledema"] = extracted_is_papilledema
           
          
        except Exception as err:
            prediction_text = "extract failure"
        finally:
            prediction_list.append(extracted_referral_content)
            print("is_papilledema_prediction: " + extracted_is_papilledema + "\n")
            print("referral_content_prediction: " + extracted_referral_content + "\n")
            print("ground_truth: " + reference_text)
            print("=============================")

        bleu_score = sacrebleu.corpus_bleu([extracted_referral_content], [[reference_text]])
        bleu_score_list.append(bleu_score.score)
        single_test["bleu"] = bleu_score.score

        rouge_score = rouge.get_scores(extracted_referral_content, reference_text)
        rouge_score_list.append(rouge_score)
        
        
        
    # 创建 CSV 文件
    csv_data = []

    for single_test in test_data_json:
        csv_data.append({
            # "id": single_test["id"],
            # "name": single_test["name"],
            "instruction": single_test["instruction"],
            "whole_letter": single_test["whole_letter"],
            "referral_content": single_test["referral_content"],
            "predict_referral_content": single_test["predict_referral_content"],
            "bleu": single_test["bleu"],
            "is_papilledema": single_test["predict_is_papilledema"],
        })

    # 创建 DataFrame
    df = pd.DataFrame(csv_data)

    # 保存为 CSV 文件
    df.to_csv(csv_file_path, index=False, encoding='utf-8')

    print(f"CSV file has been saved to {csv_file_path}")
        
    return bleu_score_list
              

In [42]:
bleu_score_list = evaluate_testjsonl_with_gemma("../test.jsonl", "./gemma_originModel_result.csv")

is_papilledema_prediction: False

referral_content_prediction: The reason for this referral is to further evaluate and manage Ms. Jane Doe's progressively worsening cardiopulmonary symptoms. Given her clinical presentation, further investigation is essential to rule out possible cardiac etiologies, including congestive heart failure or valvular disease.

ground_truth: The reason for this referral is to further evaluate and manage Ms. Jane Doe's progressively worsening cardiopulmonary symptoms. Given her clinical presentation, further investigation is essential to rule out possible cardiac etiologies, including congestive heart failure or valvular disease.
is_papilledema_prediction: False

referral_content_prediction: Mr. Davis has presented with significant neck pain and dropped head syndrome without any neurological abnormalities. Despite physical therapy, including cervical spine traction and extensor muscle training, there has been no improvement.

ground_truth: Mr. Davis has presen

In [43]:
def analyze_predict_data(bleu_score_list):
    # 统计大于100的个数
    count_gt_100 = sum(1 for score in bleu_score_list if score >= 100)

    # 统计大于70的个数
    count_gt_70 = sum(1 for score in bleu_score_list if score > 70)

    prob_gt_100 = count_gt_100 / len(bleu_score_list)
    prob_gt_70 = count_gt_70 / len(bleu_score_list)
    average_score = sum(bleu_score_list) / float(len(bleu_score_list))

    print(f"分数大于100的个数：{count_gt_100}, 占所有数据的百分比为： {prob_gt_100}")
    print(f"分数大于70的个数：{count_gt_70}, 占所有数据的百分比为： {prob_gt_70}")
    print(f"bleu平均分数: {average_score}")

In [44]:
analyze_predict_data(bleu_score_list)

分数大于100的个数：81, 占所有数据的百分比为： 0.5785714285714286
分数大于70的个数：90, 占所有数据的百分比为： 0.6428571428571429
bleu平均分数: 76.30147634147701


In [9]:
import sacrebleu
from rouge import Rouge
import pandas as pd
import re
import json


# def extract_referral_content(prediction_text):
#     try:
#         # 使用正则表达式从生成的文本中提取 referral_content 的值
#         match = re.search(r'{"referral_content":\s*"([^"]+)"', prediction_text)
#         if match:
#             return match.group(1)
#         else:
#             return "extract failure"
#     except Exception as err:
#         return "extract failure"

    
def print_testjsonl_with_gemma(reference_path):
    
    test_data_json = []
    
    with open(reference_path, 'r', encoding='utf-8') as f:
        for line in f:
              test_data_json.append(json.loads(line.strip()))
    
    rouge_score_list = []
    bleu_score_list = []
              
    rouge = Rouge()
              
    prediction_list = []
              
    for single_test in test_data_json:
        instruction = single_test["instruction"]
        whole_letter = single_test["whole_letter"]
        referral_content = single_test["referral_content"]
        prompt = f"{instruction}\n\n###\n\n{whole_letter}\n\n###"
        response = origin_model.predict({'inputs': prompt, 'parameters': {'max_new_tokens': 256}})
       
        reference_text = referral_content
        # print(response[0])
        try:
            # print("response: " + response)
            # response = reponse[0] if isinstance(response, list) else response
            # print("response: " + response)
            # prediction_dict = json.loads(response["generated_text"].strip())
            prediction_text = response[0]["generated_text"].strip()
            print(prediction_text + "\n")
            
            # extracted_referral_content = extract_referral_content(prediction_text)
            # print(extracted_referral_content)
            
#             single_test["predict_referral_content"] = extracted_referral_content
           
          
        except Exception as err:
            prediction_text = "extract failure"
        finally:
            # prediction_list.append(extracted_referral_content)
            # print("prediction: " + extracted_referral_content + "\n")
            print("ground_truth: " + reference_text)
            print("=============================")

#         bleu_score = sacrebleu.corpus_bleu([extracted_referral_content], [[reference_text]])
#         bleu_score_list.append(bleu_score.score)
#         single_test["bleu"] = bleu_score.score

#         rouge_score = rouge.get_scores(extracted_referral_content, reference_text)
#         rouge_score_list.append(rouge_score)
        
        
        
#     # 创建 CSV 文件
#     csv_data = []

#     for single_test in test_data_json:
#         csv_data.append({
#             "id": single_test["id"],
#             "name": single_test["name"],
#             "instruction": single_test["instruction"],
#             "whole_letter": single_test["whole_letter"],
#             "referral_content": single_test["referral_content"],
#             "predict_referral_content": single_test["predict_referral_content"],
#             "bleu": single_test["bleu"],
#         })

#     # 创建 DataFrame
#     df = pd.DataFrame(csv_data)

#     # 保存为 CSV 文件
#     df.to_csv(csv_file_path, index=False, encoding='utf-8')

#     print(f"CSV file has been saved to {csv_file_path}")
        
#     return bleu_score_list
              
    

In [10]:
print_testjsonl_with_gemma("../test.jsonl")

Synonyms: ["Papilledema","Swollen discs","Indistinct margins","Blurred disc margins","Suspicious discs","Disc swelling","Optic nerve swelling"]

PseudoSynonyms: ["Pseudopapilledema","Drusen","Tilted disc","Anomalous discs"]


Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.

Instruction : please extract the referral content from the following referral letter  separeted by ###. 

output your result directly in format: "is_Papilledema": boolean, "referral_content": "".

Rule For is_Papilledema : If the referral letter contains one of words in Synonyms, then is_Papilledema = true; If the letter contains words in PseudoSynonyms or doesn't contain words in Synonyms, then is_Papilledema = False.

Rule For referral_content : this content should be a whole paragraph which tells Patient need referral. If the referral_letter contains this content, you should include it. If the letter doesn't contain related information, then it should be

In [25]:
# def analyze_predict_data(bleu_score_list):
#     # 统计大于100的个数
#     count_gt_100 = sum(1 for score in bleu_score_list if score >= 100)

#     # 统计大于70的个数
#     count_gt_70 = sum(1 for score in bleu_score_list if score > 70)

#     prob_gt_100 = count_gt_100 / len(bleu_score_list)
#     prob_gt_70 = count_gt_70 / len(bleu_score_list)
#     average_score = sum(bleu_score_list) / float(len(bleu_score_list))

#     print(f"分数大于100的个数：{count_gt_100}, 占所有数据的百分比为： {prob_gt_100}")
#     print(f"分数大于70的个数：{count_gt_70}, 占所有数据的百分比为： {prob_gt_70}")
#     print(f"bleu平均分数: {average_score}")

In [26]:
analyze_predict_data(bleu_score_list)

分数大于100的个数：6, 占所有数据的百分比为： 0.2857142857142857
分数大于70的个数：6, 占所有数据的百分比为： 0.2857142857142857
bleu平均分数: 55.95564228861558
